In [1]:
import tensorflow as tf
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd

import random

from sklearn.metrics import silhouette_score

from sklearn.preprocessing import StandardScaler
from itertools import combinations

from collections import Counter

We load the dataset, normalise it to [-1, 1] and flatten it

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
n_train = x_train.shape[0]
n_test = x_test.shape[0]

# normalise to [-1, 1] so that PCA is not swayed by features with bigger value ranges.
x_train, x_test = x_train/127.5 - 1, x_test/127.5 - 1

# each pixel is a feature. num of pixels = height x width
nb_features = np.prod(x_train.shape[1:])

x_train.resize((n_train, nb_features))
x_test.resize((n_test, nb_features))

### Task 1. Data visualisation (10 points)

In [ ]:
# whe can perform PCA since data is already standardised
pca = PCA(n_components=2)
x_train_pca = pca.fit_transform(x_train)

In [ ]:
plt.figure(figsize=(8, 5))
# Plot all classes iteratively, on same plot
for digit in range(10):
    plt.scatter(x_train_pca[y_train == digit, 0], x_train_pca[y_train == digit, 1], label=f'Class {digit}', alpha=0.7)

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles)) 

plt.legend(by_label.values(), by_label.keys())
plt.title('PCA of Dataset with 10 Classes')
plt.xlabel('PC 1')
plt.ylabel('PC 2')
plt.show()


We get a general idea of the whole dataset but we have to to brake it into smaller part in order to understand it better

In [ ]:
# all pairs of classes
class_pairs = list(combinations(range(10), 2))

# set up plot table shape
fig, axes = plt.subplots(len(class_pairs) // 5 + (len(class_pairs) % 5 != 0), 5, figsize=(5, 8.5))

axes = axes.flatten()

total_sil_score = 0

# for all pairs of digits, take their pca data that we calculated before, plot them and get their silhouette score
for idx, (class1, class2) in enumerate(class_pairs):
    ax = axes[idx]

    cond = (y_train == class1) + (y_train == class2)
    x_train_pca_bin = x_train_pca[cond, :]
    y_train_pca_bin = y_train[cond] * 1

    y_train_pca_bin = np.where(y_train_pca_bin == class1, -1, 1)

    # this calculates the similarity of the cluster's samples which gives a hint to their separability. 
    # Values range from [0 to 1] for datasets with no misclassifications. Higher is better
    sil_score = silhouette_score(x_train_pca_bin, y_train_pca_bin)
    total_sil_score += sil_score
    
    # plot first and second component, for one class and then the other
    ax.scatter(x_train_pca_bin[y_train_pca_bin == -1, 0], x_train_pca_bin[y_train_pca_bin == -1, 1], alpha=0.7, s=10, label=f'Class {class1}')
    ax.scatter(x_train_pca_bin[y_train_pca_bin == 1, 0], x_train_pca_bin[y_train_pca_bin == 1, 1], alpha=0.7, s=10, label=f'Class {class2}')
    
    # Making plot less cluttered
    ax.set_title(f'{class1} vs {class2}', fontsize=6)
    ax.set_aspect('equal')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')

# Remove empty subplots
for idx in range(len(class_pairs), len(axes)):
    fig.delaxes(axes[idx])

# Reuce white space
plt.subplots_adjust(wspace= 0.02, hspace=0.33)
plt.show()

print('Average silhouette score is:', round(total_sil_score/len(class_pairs), 3))

No pair of classes seems easily seperable. We will perform PCA in pairs in order to reduce the noise and influence of other classes.

In [ ]:
# function that accepts two digits and returns the corresponding slices of train data and labels
def digit_pairs(x, y, digit1, digit2):
    if digit1 == digit2:
        print('Same digit given twice')
        raise ValueError
    
    cond = (y==digit1) + (y==digit2)
    x_binary = x[cond, :]
    y_binary = y[cond].astype(int)
    y_binary = np.where(y_binary == digit1, -1, 1)

    return x_binary, y_binary

In [ ]:
# Same as before, but now PCA is done on each slice of digit pairs, individually
class_pairs = list(combinations(range(10), 2))

fig, axes = plt.subplots(len(class_pairs) // 5 + (len(class_pairs) % 5 != 0), 5, figsize=(4.5, 8))
fig.patch.set_facecolor('lightblue')

axes = axes.flatten()

total_sil_score = 0

pca2 = PCA(n_components=2)
for idx, (class1, class2) in enumerate(class_pairs):
    ax = axes[idx]
    x_train_bin, y_train_bin = digit_pairs(x_train, y_train, class1, class2)
    x_train_bin_pca = pca2.fit_transform(x_train_bin)

    sil_score = silhouette_score(x_train_bin_pca, y_train_bin)
    total_sil_score += sil_score
    
    # Plot
    ax.scatter(x_train_bin_pca[y_train_bin == -1, 0], x_train_bin_pca[y_train_bin == -1, 1], alpha=0.7, s=10, label=f'Class {class1}')
    ax.scatter(x_train_bin_pca[y_train_bin == 1, 0], x_train_bin_pca[y_train_bin == 1, 1], alpha=0.7, s=10, label=f'Class {class2}')
    
    # Making plot less cluttered
    ax.set_title(f'{class1} vs {class2}', fontsize=6)
    ax.set_aspect('equal')
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel('')
    ax.set_ylabel('')

# Remove empty subplots
for idx in range(len(class_pairs), len(axes)):
    fig.delaxes(axes[idx])

# Reuce white space
plt.subplots_adjust(wspace= 0.02, hspace=0.33)
plt.show()

print('Average silhouette score is:', round(total_sil_score/len(class_pairs), 3))

We see that the silhouette score has increased, indicating that the samples of each class are better formed than before. Indeed, a visual inspection reveals that most pairs now seem more easily seperable. No pair appears perfectly linearily seperable but a lot (3,4), (3,7), (6, 9) appear more distinct. This concludes Part 1. Next, we will try to train a perceptron to be able to seperate two classes.

### Task 2. Perceptrons (10 points)

Training a perceptron can be broken down into two components: 1) getting the output of the perceptron and 2) using that output to guide the update of its parameters.

The way a perceptron or a neuron works, is that it receives an arbitary number of inputs/stimulations $x = [x_1, x_2, ...]$. These inputs are scaled by a similarily sized weight vector $w^T = [w_1, w_2, ...]$. Thats the perceptron's way of assigning an importance and meaning to each input. Finaly, the sum of the weighted inputs is compared agains a threshold $b$, and a decision is taken based on the result. This serves the puprose if intruducing a non-linearity which mimics the way decisions are often made. (e.g, the higher the temperatuer the more likely it is people will go for a swim, but going for a swim is a binary situation as people generaly do not travel halfway to the swimming pool because the temperature is halfway to heatwave.)


The function `predict(x, w b)` implements these calculations and returns the decision

In [ ]:
# calculates dot product of input-x with weights-w and compares to threshold. Returns the result of the comparison which is the final decision.
def predict(x, w, b):
    weighted_sum = np.dot(x, w)
    result = 1 if weighted_sum > b else -1
    return result

This step exposed two kinds of parameters of the perceptron that are not given by the dataset: the weight vector $w$ and the threshold or bias $b$. As these can be set arbitarily, the problem of producing a neuron that pefrorms better than a random chooser can be seen as working out a method that modifies the customisable parameters so that the produce the right output.

One approach for achieving this is as follows:\
a) get one of the training samples and use it to produce a prediction. If the prediction is correct no action needs to be taken. (In the extreme case where all predictions are correct, modifying the parameters can only make the predictor worse)\
b) if the prediction is wrong, the parameters will be changed with two things in mind: 1. The output must change to the direction of the target label. If the target is $1$ and the prediction was $-1$ then the dot product of the input and the weights must increase so that it grows closer to the target, and vice versa. We can get the direction of desired change with $(target - prediction)$. 2. Of all the parameters of the model_deeper_deeper, the ones that are responsible for the mistake must change the most. Naturally, changing the weights of features that are irrelevant to this misclassification will not helpe prevent it in the future, and could undo the work done on these features. We can scale the change to the neurons weights based on their relevance to the misclassification by multiplying with the sample, as by definition, this sample has high values of the features that correspond to the weights that must be changed.
Finaly, a hyperparameter can be used to modify the level of change at each update. Larger values of this learning rate can make initial approach to the local minimum faster but as the distance to it decreases, it can force it to overshoot back and forth above it, preventing improvement. This can be avoided with smaller learning rates at the cost of convergance speed, which must be kept within reasonable times.

These form the core of the training logic. The function `optimise(x, y)` prepares the ground for training the perceptron by first randomly initialising its parameters, while also setting up additional parameters for keeping track of the error and the training iteration. \
The model iterates through all of the samples, producing a prediction and adjusting the weights and bias accordingly.\
A termination condition ensures that the algorithm stops upon reaching a satisfying results, or failing to reach one within the specified time frame.\

In [ ]:
# applyis the correction process described above to optimise a given set of parameters w and b
def optimize(x, w, b, y):
    iteration = 0
    mse = np.inf
    # found in practice to work well
    learning_rate = 0.002
    mses = []
    while (iteration <= 1000) & (mse > 1e-3):
        mse, fp, fn = 0, 0, 0
        predictions  = []
        for sample, target in zip(x, y):
            prediction = predict(sample, w, b)
            predictions.append(prediction)
            if prediction != target:
                if prediction > target:
                    fp += 1
                else:
                    fn += 1
                w = w + learning_rate*(target-prediction)*sample
                # the theshold for the dot product is not scaled based on the input, so no *sample
                b = b + learning_rate*(target-prediction)

        iteration += 1
        # average of squared error
        mse = sum((predictions - y)**2)/len(y)
        mses.append(mse)
        accuracy = round(np.sum(predictions == y)/len(y), 4)
        print("Iteration:", iteration, 'with MSE:', round(mse, 4), ', Accuracy:', accuracy, ', False positives:', fp, ', False Negatives:', fn)

    return predictions, w, b, accuracy, mses

# wraps around optimise to provide randomly initialised parameters and the data related to the two digits of interest
def init_and_train(x, y, digit1, digit2):
    x_train_binary, y_train_binary = digit_pairs(x, y, digit1, digit2)
    m, n = x_train_binary.shape
    w = np.random.uniform(-1, 1, n)
    b = np.random.uniform(-1, 1)
    
    return optimize(x_train_binary, w, b, y_train_binary)

preds, weights, bias, accuracy, mses = init_and_train(x_train, y_train, 8, 9)

In [ ]:
# Plot of MSE vs number of iteration
plt.figure(figsize=(15, 4))
plt.plot(mses)
plt.xlabel('Iteration')
plt.ylabel('MSEs')
plt.title('Plot of Mean Square Classification Error')
plt.show()

In [ ]:
# used the model to produce prediction for the test samples, which allows for the test data accuracy to be calculated
def test_data_check(x, y, digit1, digit2, weights, bias):
    x_test_binary, y_test_binary = digit_pairs(x, y, digit1, digit2)
    predictions = []
    for sample in x_test_binary:
        predictions.append(predict(sample, weights, bias))

    accuracy = round(np.sum(predictions == y_test_binary)/len(y_test_binary), 4)
    mse = sum((predictions - y_test_binary)**2)/len(y)
    return accuracy, mse

print(test_data_check(x_test, y_test, 8, 9, weights, bias))

We can edit the optimise function to keep track of the test MSE throughout the training

In [ ]:
def optimize(x, w, b, y):
    iteration = 0
    mse = np.inf
    learning_rate = 0.002
    mses = []
    test_mses = []
    while (iteration <= 1000) & (mse > 1e-3):
        mse, fp, fn = 0, 0, 0
        predictions  = []
        for sample, target in zip(x, y):
            prediction = predict(sample, w, b)
            predictions.append(prediction)
            if prediction != target:
                if prediction > target:
                    fp += 1
                else:
                    fn += 1
                w = w + learning_rate*(target-prediction)*sample
                b = b + learning_rate*(target-prediction)

        iteration += 1
        mse = sum((predictions - y)**2)/len(y)
        mses.append(mse)
        # track mse for test dataset as well
        _, test_mse = test_data_check(x_test, y_test, 8, 9, w, b)
        test_mses.append(test_mse)
        accuracy = round(np.sum(predictions == y)/len(y), 4)
        print("Iteration:", iteration, 'with MSE:', round(mse, 4), ', Accuracy:', accuracy, ', False positives:', fp, ', False Negatives:', fn)

    return predictions, w, b, accuracy, mses, test_mses

In [ ]:
preds, weights, bias, accuracy, mses, test_mses = init_and_train(x_train, y_train, 8, 9)

In order to evaluate the performance of the classifier we need to know the balance of the classes. Below we plot the class distribution for train and test datasets

In [ ]:
y_train_counts = Counter(y_train)
y_test_counts = Counter(y_test)

axis_points = sorted(set(y_train_counts.keys()).union(y_test_counts.keys()))

frequencies1 = [y_train_counts[item] for item in axis_points]
frequencies2 = [y_test_counts[item] for item in axis_points]

bar_width = 0.4

r1 = [x - bar_width/2 for x in range(len(axis_points))]
r2 = [x + bar_width/2 for x in range(len(axis_points))]

plt.bar(r1, frequencies1, width=bar_width, color='skyblue', label='Train')
plt.bar(r2, frequencies2, width=bar_width, color='orange', label='Lest')
plt.xticks(range(len(axis_points)), axis_points)
plt.xlabel("Digit")
plt.ylabel("Frequency")
plt.title("Training and Test Histogram")
plt.legend()
plt.show()

We see that the classes are close to an equal representation, thus the baseline accuracy and MSE are $10\%$ and $1/10*(2^2)*(10-1) = 3.6$

In [ ]:
y_train_counts = Counter(y_train)
y_test_counts = Counter(y_test)

all_items = sorted(set(y_train_counts.keys()).union(y_test_counts.keys()))

frequencies1 = [y_train_counts[item] for item in all_items]
frequencies2 = [y_test_counts[item] for item in all_items]

bar_width = 0.4

r1 = [x - bar_width/2 for x in range(len(all_items))]
r2 = [x + bar_width/2 for x in range(len(all_items))]

plt.bar(r1, frequencies1, width=bar_width, color='skyblue', label='List 1')
plt.bar(r2, frequencies2, width=bar_width, color='salmon', label='List 2')
plt.xticks(range(len(all_items)), all_items)
plt.xlabel("Items")
plt.ylabel("Frequency")
plt.title("Overlayed Histogram of Two Lists")
plt.legend()
plt.show()

In [ ]:
# Plot of train dataset MSE against test dataset MSE vs number of iteration
plt.figure(figsize=(15, 4))
plt.plot(mses)
plt.plot(test_mses)
plt.xlabel('Iteration')
plt.ylabel('MSEs')
plt.title('Plot of Mean Square Classification Error')
plt.show()

We see that while the training MSE keeps decreasing, the test MSE has its minimus at around the 20 iterations point, after which is starts to slowly climp, suggesting overfitting. Given that, training should be stopped at around 20 iterations.

Given that the 2d ndarray `weights` has similar structure with our samples, we can reverse the process that turned them from images to arrays to visualise what our perceptron has learned

In [ ]:
digit = weights.reshape(28, 28)

In [ ]:
def print_digit(sample_no):
    digit = sample_no.reshape(28, 28)

    plt.imshow(digit, cmap='gray')
    plt.axis('off')
    plt.show()

In [ ]:
print_digit(digit)

Here, grey values represent pixes towards which the model has learned to be indifferent (like the edges of the image which usually are not enough to differentiate any digits). With high or low brighness are represented pixels that the model has found sway the prediction towards one of the two classes. While not obvious, we can perhaps see how these pixels are part of the handwriten digit for only one of the classes. \
We can see which pixels would make good features for distinguishing of the two classes by calculating the difference between the pixels that are usually part of the one digit and the ones that are usually part of the other digit, as follows:

In [ ]:
x_8 = x_train[y_train == 8]
x_9 = x_train[y_train == 9]

average_8 = np.mean(x_8, axis=0)
average_9 = np.mean(x_9, axis=0)

dif = average_8 - average_9
print_digit(dif)

There are some similarities, at mid-left mostly, but there does not seem to be too much agreement between the two pictures. This is probably due to the fact that the perceptron learns to use whichever pixels help it classify correctly the samples it failes at, so this could be specific pixels for specific samples, not only the ones that are usefull for the general case. Their weights will get high values, even if they are only usefull rarely. 

eeee prepei na kaneis train

In [ ]:
random_pairs = [random.sample(range(10), 2) for _ in range(5)]
accuracies = []
for pair in random_pairs:
    preds, weights, bias, accuracy, mses, test_mses = init_and_train(x_train, y_train, pair[0], pair[1])
    accuracies.append(accuracy)


In [ ]:
df = pd.DataFrame({'Digit Pair':random_pairs, 'Accuracy':accuracies})
df

We see that the accuracy scores vary depending on the pair. A visual inspection reveals that pairs with similar overal shape or many similar parts have lower accuracy, for the obvious reasons. We could say that the fewer/shorter lines one needs to draw in order to make one digit look like the other, the lower the accuracy score as noise from the hand drawing can more easily create these lines, leading to a misclassification.

### Task 3. Multi-Layer Perceptron (10 points)

We will now use create a MLP Neural Network with the given architecture specification.

In [2]:
import tensorflow as tf
import os
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

mnist = tf.keras.datasets.mnist
import datetime

The model will perform multiclass classification so we will convert the labels from integers to one-hot encoding, paired with the categorical_crossentropy loss function

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train/255
y_train = np.eye(10)[y_train]
X_test = X_test/ 255
y_test = np.eye(10)[y_test]

n_train = X_train.shape[0]
n_test = X_test.shape[0]

# each pixel is a feature. num of pixels = height x width
nb_features = np.prod(X_train.shape[1:])

X_train.resize((n_train, nb_features))
X_test.resize((n_test, nb_features))

In [ ]:
model = Sequential()

# We implement the given specifications
model.add(Dense(1000, input_shape=(X_test.shape[1],), activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer=Adam(learning_rate=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

# We will use tensorboard to log the results.
log_dir = os.path.join("c:/Users/Emmts/OneDrive/Desktop/University of Bath/Phase 3 - Unit 9 - Foundations and Frontiers of Machine Learning/FFoML_GA2/logs", 'standard')
print(log_dir)
tensorboard_callback = TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=True,
    write_images=True,
    update_freq='epoch'
)


history = model.fit(X_train, y_train, 
                    epochs=2,
                    batch_size=256,
                    validation_data = (X_test, y_test),     # we do not perform a validation split, instead testing on the test dataset to produce the desired plots
                    callbacks=[tensorboard_callback])

test_loss, test_acc = model.evaluate(X_test, y_test)

c:/Users/Emmts/OneDrive/Desktop/University of Bath/Phase 3 - Unit 9 - Foundations and Frontiers of Machine Learning/FFoML_GA2/logs\standard
Epoch 1/2
235/235 [==============================] - 6s 22ms/step - loss: 0.2268 - accuracy: 0.9324 - val_loss: 0.1030 - val_accuracy: 0.9663
Epoch 2/2
313/313 [==============================] - 1s 4ms/step - loss: 0.0762 - accuracy: 0.9770


In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1000)              785000    
                                                                 
 dense_1 (Dense)             (None, 1000)              1001000   
                                                                 
 dense_2 (Dense)             (None, 10)                10010     
                                                                 
Total params: 1,796,010
Trainable params: 1,796,010
Non-trainable params: 0
_________________________________________________________________


We notice very good results, owning to the almost 2 million parameters. We will test various architectures to draw conclusions about the relations between number of layer, number of neurons and accuracy

In [6]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir "c:/Users/Emmts/OneDrive/Desktop/University of Bath/Phase 3 - Unit 9 - Foundations and Frontiers of Machine Learning/FFoML_GA2/logs"

In [8]:
# This function returns a TensorBoard object with parameterised location reflecting different architectures
def tb_callback_setter(subdir):
    log_dir = os.path.join("c:/Users/Emmts/OneDrive/Desktop/University of Bath/Phase 3 - Unit 9 - Foundations and Frontiers of Machine Learning/FFoML_GA2/logs", subdir)

    tensorboard_callback = TensorBoard(
        log_dir=log_dir,
        histogram_freq=1,
        write_graph=True,
        write_images=True,
        update_freq='epoch'
        )
    
    return tensorboard_callback

In [9]:
# we repeat the experiment for various depths to investigate how they affect the models characteristics
# we reduce the width of the layers to stay within reasonable training times, also with 1000 the model performs well enough so the added layers have no room for improvement
def create_and_train(num_of_layers, epochs, learning_rate, batch_size):

    model = Sequential()

    model.add(Dense(500, input_shape=(X_test.shape[1],), activation='relu'))

    for _ in range(num_of_layers - 1):
        model.add(Dense(500, activation='relu'))

    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=Adam(learning_rate=learning_rate), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])
    
    tensorboard_callback = tb_callback_setter(f'{num_of_layers}_Layers')

    history = model.fit(X_train, y_train, 
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data = (X_test, y_test),
                    callbacks=[tensorboard_callback])
    
    test_loss, test_acc = model.evaluate(X_test, y_test)
    params = model.count_params()

    return model, test_acc, test_loss, params

In [10]:
numbers_of_Layers = [3, 4, 6, 8]
models = []
mlp_accuracies, mlp_losses, mlp_params = [], [], []

for layers in numbers_of_Layers:
    model, acc, loss, params = create_and_train(layers, 2, 0.001, 64)
    models.append(model)
    mlp_accuracies.append(acc)
    mlp_losses.append(loss)
    mlp_params.append(params)

Epoch 1/2
938/938 [==============================] - 7s 7ms/step - loss: 0.1982 - accuracy: 0.9391 - val_loss: 0.1092 - val_accuracy: 0.9665
Epoch 2/2
313/313 [==============================] - 1s 3ms/step - loss: 0.0744 - accuracy: 0.9757
Epoch 1/2
938/938 [==============================] - 8s 8ms/step - loss: 0.2101 - accuracy: 0.9372 - val_loss: 0.1209 - val_accuracy: 0.9628
Epoch 2/2
313/313 [==============================] - 2s 6ms/step - loss: 0.0896 - accuracy: 0.9751
Epoch 1/2
938/938 [==============================] - 11s 10ms/step - loss: 0.2374 - accuracy: 0.9302 - val_loss: 0.1603 - val_accuracy: 0.9544
Epoch 2/2
313/313 [==============================] - 1s 3ms/step - loss: 0.1002 - accuracy: 0.9727
Epoch 1/2
938/938 [==============================] - 13s 12ms/step - loss: 0.2647 - accuracy: 0.9242 - val_loss: 0.1440 - val_accuracy: 0.9616
Epoch 2/2
313/313 [==============================] - 3s 9ms/step - loss: 0.1280 - accuracy: 0.9694


In [ ]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
n_train = x_train.shape[0]
n_test = x_test.shape[0]
print(f"Training images {n_train}, Test images {n_test}")